# Seminar Example: Impulsively Heated AR
Setup a proof of concept for the Space Physics Seminar at Rice University. This will be an AR heated randomly by several nanoflares with 500 loops. We will image it with both SDO and EIS.

In [3]:
import os
import glob
import logging
import subprocess
logging.basicConfig(level=logging.INFO)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.ticker import LogFormatter
import astropy.units as u
from sunpy.map import Map,MapCube

import synthesizAR
from synthesizAR.model_ext import EbtelInterface,calculate_free_energy
from synthesizAR.instruments import InstrumentHinodeEIS,InstrumentSDOAIA

%matplotlib inline

 using cli
 using CLI for selections
 reading chiantirc file


/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
ar_root = '/data/datadrive2/ar_viz/seminar_poc/'

First extrapolate the field and trace the loops.

In [5]:
crop = (u.Quantity([-400,-140]*u.arcsec),u.Quantity([320,540]*u.arcsec))
resample = u.Quantity([100,100]*u.pixel)
field = synthesizAR.Skeleton('/home/wtb2/sunpy/data/hmi_m_45s_2010_06_19_03_06_00_tai_magnetogram.0.fits',
                             crop=crop,resample=resample)

/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/sunpy/image/transform.py:130: RuntimeWarning: Setting NaNs to 0 for higher-order scikit-image rotation
  RuntimeWarning)


In [6]:
zshape=50
zrange=u.Quantity([0.,150.]*u.arcsec)
field.extrapolate_field(zshape,zrange)

True


/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/numba/decorators.py:24: DeprecationWarning: autojit is deprecated, use jit instead which now performs the same functionality
  "the same functionality", DeprecationWarning)
yt : [INFO     ] 2017-02-26 02:45:58,021 Parameters: current_time              = 0.0
yt : [INFO     ] 2017-02-26 02:45:58,023 Parameters: domain_dimensions         = [96 96 46]
yt : [INFO     ] 2017-02-26 02:45:58,024 Parameters: domain_left_edge          = [ -2.89511786e+10   2.37893407e+10   4.40182588e+08]
yt : [INFO     ] 2017-02-26 02:45:58,026 Parameters: domain_right_edge         = [ -1.06601257e+10   3.93100982e+10   1.05643821e+10]
yt : [INFO     ] 2017-02-26 02:45:58,027 Parameters: cosmological_simulation   = 0.0


In [7]:
field.extract_streamlines(500,loop_length_range=[1e+9,1e+11]*u.cm)

INFO:Skeleton:Tracing fieldlines
yt : [INFO     ] 2017-02-26 02:45:58,354 AMRKDTree rebuilt, Final Volume: 2.874169e+30
Streamlining: 100%|██████████| 500/500 [00:01<00:00, 330.06it/s]
yt : [INFO     ] 2017-02-26 02:46:06,663 AMRKDTree rebuilt, Final Volume: 2.874169e+30
Streamlining: 100%|██████████| 124/124 [00:00<00:00, 341.58it/s]
yt : [INFO     ] 2017-02-26 02:46:08,842 AMRKDTree rebuilt, Final Volume: 2.874169e+30
Streamlining: 100%|██████████| 30/30 [00:00<00:00, 307.82it/s]
yt : [INFO     ] 2017-02-26 02:46:09,502 AMRKDTree rebuilt, Final Volume: 2.874169e+30
Streamlining: 100%|██████████| 6/6 [00:00<00:00, 162.60it/s]
yt : [INFO     ] 2017-02-26 02:46:09,779 AMRKDTree rebuilt, Final Volume: 2.874169e+30
Streamlining: 100%|██████████| 2/2 [00:00<00:00, 142.96it/s]


In [8]:
field.make_loops()

Then run the hydrodynamic simulations.

In [9]:
class NanoflareStormModel(object):
    def __init__(self,heating_options):
        self.heating_options = heating_options
        np.random.seed()
        
    @property
    def number_events(self):
        return 1
    
    def calculate_event_properties(self,loop):
        available_energy = calculate_free_energy(loop.field_aligned_coordinate,loop.field_strength,
                                                 stress_level=self.heating_options['stress_level'])
        rate = 2.0*available_energy/(self.number_events*(2.0*self.heating_options['duration'] 
                                                         - self.heating_options['duration_rise'] 
                                                         - self.heating_options['duration_decay']))
        #distribute start times randomly between 0 and total_time - duration
        tsr = np.random.rand(self.number_events)*(self.base_config['total_time'] - self.heating_options['duration'])
        ter = tsr + self.heating_options['duration_rise']
        tsd = ter + (self.heating_options['duration'] 
                     - self.heating_options['duration_rise'] 
                     - self.heating_options['duration_decay'])
        ted = tsr + self.heating_options['duration']
        return {'magnitude':np.array([rate]),'rise_start':tsr,'rise_end':ter,'decay_start':tsd,'decay_end':ted}

In [10]:
heating_options = {
    'duration':200.0,
    'duration_rise':100.0,
    'duration_decay':100.0,
    'stress_level':0.15,
}
heating_model = NanoflareStormModel(heating_options)
ih = synthesizAR.util.InputHandler(os.path.join(os.environ['RESEARCH_DIR'],
                                                'ebtelPlusPlus/config/ebtel.example.cfg.xml'))
base_config = ih.lookup_vars()
base_config['c1_cond0'] = 6.0
base_config['use_adaptive_solver'] = True
base_config['tau'] = 1.0
base_config['adaptive_solver_error'] = 1e-12
base_config['adaptive_solver_safety'] = 0.01
base_config['total_time'] = 5000.0
ebtel_plug = EbtelInterface(base_config,heating_model,
                            os.path.join(ar_root,'hydro_config'),
                            os.path.join(ar_root,'hydro_results'))

In [11]:
field.configure_loop_simulations(ebtel_plug)

In [12]:
for loop in field.loops:
    subprocess.call([os.path.join(os.environ['RESEARCH_DIR'],'ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

In [13]:
field.load_loop_simulations(ebtel_plug,savefile=os.path.join(ar_root,'loop_parameters.h5'))

In [14]:
field.save(savedir=os.path.join(ar_root,'checkpoint'))